In [1]:
import matplotlib.colors as mcolors
import matplotlib.animation as ma
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from itertools import product
import pandas as pd
import numpy as np
import numba as nb
import imageio
import os
import shutil

randomSeed = 100

%matplotlib inline
%config InlineBackend.figure_format = "retina"

new_cmap = mcolors.LinearSegmentedColormap.from_list(
    "new", plt.cm.jet(np.linspace(0, 1, 256)) * 0.85, N=256
)

@nb.njit
def colors_idx(phaseTheta):
    return np.floor(256 - phaseTheta / (2 * np.pi) * 256).astype(np.int32)

import seaborn as sns

sns.set_theme(
    style="ticks", 
    font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['animation.ffmpeg_path'] = "/opt/conda/bin/ffmpeg"

from main import *
from multiprocessing import Pool
import pandas as pd

colors = ["#403990", "#80A6E2", "#FBDD85", "#F46F43", "#CF3D3E"]
cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors)
cmap_r = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors[::-1])

SAVE_PATH = r"./data"

In [13]:
omegaMins = np.linspace(0.1, 3, 21)
phaseLags = np.linspace(-1, 1, 21) * np.pi
randomSeed = 10  # plot phase diagram and snapshot, use random seeds (9, 10)
strengthLambda = 0.15 * 32 * 2 * 2
distanceD0 = 1
deltaOmega = 1
boundaryLength = 7
halfAgentsNum = 500
Homega = np.block([
    [np.ones((halfAgentsNum, halfAgentsNum)), np.zeros((halfAgentsNum, halfAgentsNum))],
    [np.zeros((halfAgentsNum, halfAgentsNum)), np.ones((halfAgentsNum, halfAgentsNum))]
])

SAVE_PATH = "./data"  # r"E:\MS_ExperimentData\general"

In [14]:
models = [
    MeanFieldChiralInducedPhaseLag(
        strengthLambda=strengthLambda, distanceD0=distanceD0, boundaryLength=boundaryLength,
        phaseLag=phaseLag, 
        distribution="uniform", initPhaseTheta=np.zeros(1000),
        omegaMin=omegaMin, deltaOmega=deltaOmega,
        agentsNum=1000, savePath=SAVE_PATH, dt=0.01,
        tqdm=True, overWrite=True, randomSeed=randomSeed, shotsnaps=10
    )
    for omegaMin in omegaMins
    for phaseLag in phaseLags
]

def get_state_analysis(model):
    return StateAnalysis(model)

with Pool(49) as p:
    sas = p.map(get_state_analysis, tqdm(models))

  0%|          | 0/441 [00:00<?, ?it/s]

# Calc Order Parameter $S$ and Phase Diagram

In [4]:
SvaluesMap = dict()

def calc_Svalues(sa):
    Svalues = []
    for lastTindex in np.arange(-1, -100, -10):
        positionX, _, _ = sa.get_state(index=lastTindex)

        deltaX = MeanFieldChiralInducedPhaseLag._delta_x(
            positionX, positionX[:, np.newaxis], 
            sa.model.boundaryLength, sa.model.halfBoundaryLength
        )
        Aij = MeanFieldChiralInducedPhaseLag.distance_x(deltaX) <= 0.1
        Svalues.append(((Homega * Aij).sum(axis=0) / Aij.sum(axis=0)).mean())
    return Svalues

with Pool(49) as p:
    results = p.map(calc_Svalues, tqdm(sas))

for i, sa in enumerate(sas):
    SvaluesMap[(sa.model.omegaMin, sa.model.phaseLag)] = results[i].copy()

  0%|          | 0/441 [00:00<?, ?it/s]

In [5]:
maps = pd.DataFrame(index=omegaMins, columns=phaseLags)

for i, sa in tqdm(enumerate(sas), total=len(sas)):
    Svalues = SvaluesMap[(sa.model.omegaMin, sa.model.phaseLag)]
    maps.loc[sa.model.omegaMin, sa.model.phaseLag] = np.percentile(Svalues, 30)  # np.percentile(Svalues, 50)

maps = maps.astype(float)
maps.to_csv(
    f"./tables/meanFieldOrderParameter_S_"
    f"l{strengthLambda}_d{distanceD0}_dO{deltaOmega}_rS{randomSeed}.csv"
)

  0%|          | 0/441 [00:00<?, ?it/s]

In [6]:
MvaluesMap = dict()

def calc_Svalues(sa):
    Mvalues = []
    for lastTindex in np.arange(-1, -100, -10):
        positionX, _, _ = sa.get_state(index=lastTindex)

        deltaX = MeanFieldChiralInducedPhaseLag._delta_x(
            positionX, positionX[:, np.newaxis], 
            sa.model.boundaryLength, sa.model.halfBoundaryLength
        )
        Aij = MeanFieldChiralInducedPhaseLag.distance_x(deltaX) <= 0.1
        Mvalues.append(
            np.sign(
                ((1 - Homega) * Aij).sum(axis=0)
            )
        )
    return Mvalues

with Pool(49) as p:
    results = p.map(calc_Svalues, tqdm(sas))

for i, sa in enumerate(sas):
    MvaluesMap[(sa.model.omegaMin, sa.model.phaseLag)] = results[i].copy()

  0%|          | 0/441 [00:00<?, ?it/s]

In [7]:
maps = pd.DataFrame(index=omegaMins, columns=phaseLags)

for i, sa in tqdm(enumerate(sas), total=len(sas)):
    Mvalues = MvaluesMap[(sa.model.omegaMin, sa.model.phaseLag)]
    maps.loc[sa.model.omegaMin, sa.model.phaseLag] = np.mean(Mvalues)  # np.percentile(Svalues, 50)

maps = maps.astype(float)
maps.to_csv(
    f"./tables/meanFieldOrderParameter_M_"
    f"l{strengthLambda}_d{distanceD0}_dO{deltaOmega}_rS{randomSeed}.csv"
)

  0%|          | 0/441 [00:00<?, ?it/s]